In [ ]:
import os
import json
import pandas as pd
from pydub import AudioSegment

# 시간을 밀리세컨드 단위로 변환하는 함수
def time_to_milliseconds(time_str):
    h, m, s = time_str.split(':')
    return int(h) * 3600000 + int(m) * 60000 + int(float(s) * 1000)

# 오디오 파일 분할 함수
def split_audio(audio_path, intervals):
    # 오디오 파일 로드
    audio = AudioSegment.from_file(audio_path)

    # 원본 오디오 파일의 이름에서 확장자를 제거
    base_name = os.path.splitext(os.path.basename(audio_path))[0]

    for i, (start_time, end_time) in enumerate(intervals):
        # 오디오 파일 분할
        extract = audio[start_time:end_time]

        # 분할한 오디오 파일 저장 (원본 오디오 파일의 이름을 포함)
        extract.export(f"/content/drive/MyDrive/final_dataset/{base_name}_{i}.mp3", format="mp3")

# 'wave1'과 '예능json' 디렉토리에 있는 파일들의 리스트를 얻기
wave_files = [f for f in os.listdir('/content/drive/MyDrive/wave1') if f.endswith('.wav')]
json_files = [f for f in os.listdir('/content/drive/MyDrive/예능json') if f.endswith('.json')]

# 전체 결과를 저장할 데이터프레임 생성
df_total = pd.DataFrame(columns=['sentence', 'audio_file', 'interval'])

# .wav 파일과 .json 파일의 이름을 비교
for wave_file in wave_files:
    # .wav 파일의 이름에서 확장자를 제거
    wave_file_name = os.path.splitext(wave_file)[0]

    # 같은 이름의 .json 파일이 있는지 확인
    if wave_file_name + '.json' in json_files:
        # JSON 파일 열기
        with open(f'/content/drive/MyDrive/예능json/{wave_file_name}.json', 'r', encoding='utf-8') as f:
            data = json.load(f)

        # 데이터프레임을 생성할 리스트 초기화
        df_data = []

        # 각 문장에 대한 정보를 반복해서 처리
        for i, sentence in enumerate(data['media_info']['sentence_list']):
            start = time_to_milliseconds(sentence['start'])
            end = time_to_milliseconds(sentence['end'])
            df_data.append([sentence['sentence'], f"/content/drive/MyDrive/final_dataset/{wave_file_name}_{i}.mp3", (start, end)])

        # 리스트를 데이터프레임으로 변환
        df = pd.DataFrame(df_data, columns=['sentence', 'audio_file', 'interval'])

        # 오디오 파일 분할
        split_audio(f"/content/drive/MyDrive/wave1/{wave_file}", df['interval'].values)

        # 결과를 전체 데이터프레임에 추가
        df_total = pd.concat([df_total, df])

# 전체 결과 출력
print(df_total)
